# Set run directory

In [33]:
from pathlib import Path
runDir = Path('2018-04-25_10-44-35')
runDir.exists()

True

# Update records

In [34]:
import pandas as pd
from pathlib import Path
from json import load
from os import stat

# Check for errors
if stat(runDir / 'err.txt').st_size != 0:
    raise ValueError('The error file is not empty! Check err.txt!')
if stat(runDir / 'out.txt').st_size != 0:
    raise ValueError('Something was written to standard output! Check out.txt!')

# Open records
records = pd.DataFrame()
if Path('records.csv').exists():
    records = pd.read_csv('records.csv')

with (runDir / '0.json').open('r') as recF: 
    data = load(recF)
    del data['CV sample times [s]']
    del data['CV samples']
    data['Info'] = 'SIAM' # TODO
    
    records = records.append(data, ignore_index=True)
records.tail()

,Device,Domain partitions,Exact solution,Executable name,Executable path,Info,Initial guess,Iterant,Matrix,Nodes,Processes,Processes grid,Processes per node,Rank,Relative error,Relative residual,Threads per process,Timeout [min],Timestamp,Tolerance
0,julia,"[[0, 100], [0, 100], [0, 100]]",white-noise sine,example.laplace3d_ajacobi,/gpfs/homeb/zam/tnikolov/asynchronator/build/e...,SIAM,zero,jacobi,laplace3d,1.0,1.0,"[1, 1, 1]",1.0,0.0,0.000143,1.362678e-05,256.0,2.0,2018-04-25 10:38:12,0.0001
1,julia,"[[0, 50, 100], [0, 100], [0, 100]]",white-noise sine,example.laplace3d_ajacobi,/gpfs/homeb/zam/tnikolov/asynchronator/build/e...,SIAM,zero,ajacobi,laplace3d,2.0,2.0,"[2, 1, 1]",1.0,0.0,0.000280,1.315764e-06,256.0,2.0,2018-04-25 10:39:09,0.0001
2,julia,"[[0, 50, 100], [0, 50, 100], [0, 100]]",white-noise sine,example.laplace3d_ajacobi,/gpfs/homeb/zam/tnikolov/asynchronator/build/e...,SIAM,zero,ajacobi,laplace3d,4.0,4.0,"[2, 2, 1]",1.0,0.0,0.000190,2.689691e-07,256.0,2.0,2018-04-25 10:39:37,0.0001
3,julia,"[[0, 25, 50, 75, 100], [0, 50, 100], [0, 50, 1...",white-noise sine,example.laplace3d_ajacobi,/gpfs/homeb/zam/tnikolov/asynchronator/build/e...,SIAM,zero,ajacobi,laplace3d,16.0,16.0,"[4, 2, 2]",1.0,0.0,0.002274,3.192296e-06,256.0,2.0,2018-04-25 10:44:17,0.0001
4,julia,"[[0, 25, 50, 75, 100], [0, 25, 50, 75, 100], [...",white-noise sine,example.laplace3d_ajacobi,/gpfs/homeb/zam/tnikolov/asynchronator/build/e...,SIAM,zero,ajacobi,laplace3d,32.0,32.0,"[4, 4, 2]",1.0,0.0,0.007527,1.071060e-05,256.0,2.0,2018-04-25 10:44:35,0.0001


# Save records

In [35]:
records.to_csv('records.csv', index=False)

# Read iterations record

In [36]:
# Read files
itersData = []
for runP in runDir.glob('*.json'):
    if runP.name == 'setup.json': 
        continue
    with runP.open('r') as recF: 
        data = load(recF)
        dataDf = pd.DataFrame(data['CV samples'], 
                              index=data['CV sample times [s]'],
                              columns =['Rank {}'.format(runP.name[:-5])])
        dataDf.index.name = 'Time [s]'
        itersData.append(dataDf)

itersDf = pd.concat(itersData, axis=1)
itersDf.head()

,Rank 27,Rank 5,Rank 14,Rank 19,Rank 1,Rank 20,Rank 8,Rank 25,Rank 12,Rank 17,...,Rank 18,Rank 6,Rank 24,Rank 29,Rank 11,Rank 2,Rank 16,Rank 30,Rank 9,Rank 22
Time [s],,,,,,,,,,,,,,,,,,,,,
0.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
0.001035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
0.001079,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.001204,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.001274,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Save iterations record

In [37]:
itersDf.to_csv('{}.csv'.format(runDir.name))

# Clean run directory

In [38]:
for f in runDir.glob('*'):
    f.unlink()
runDir.rmdir()